_API Reference: https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#steps_

In [1]:
!pip install sagemaker --upgrade

  Using cached sagemaker-2.231.0-py3-none-any.whl.metadata (16 kB)
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
Using cached sagemaker-2.231.0-py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 55.7 MB/s eta 0:00:0000:010:01
Using cached mock-4.0.3-py3-none-any.whl (28 kB)
  Attempting uninstall: mock
    Found existing installation: mock 5.1.0
    Uninstalling mock-5.1.0:
      Successfully uninstalled mock-5.1.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.131
    Uninstalling boto3-1.34.131:
      Successfully uninstalled boto3-1.34.131
  Attempting uninstall: sagemaker
    Found ex

In [14]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TrainingStep, ProcessingStep, CreateModelStep
from sagemaker.workflow.parameters import ParameterString
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.model_step import ModelStep
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

# 初始化 SageMaker 執行角色和區域
role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()

print(role)
print(region)
sess

arn:aws:iam::070576557102:role/service-role/AmazonSageMaker-ExecutionRole-20240512T164029
ap-northeast-1


In [7]:
# 定義參數
s3_input_data = ParameterString(name="InputData", default_value="s3://your-bucket/input-data/")
instance_type = ParameterString(name="InstanceType", default_value="ml.m5.xlarge")
model_output_path = ParameterString(name="ModelOutputPath", default_value="s3://your-bucket/model-output/")


In [8]:
# 訓練模型
estimator = Estimator(
    image_uri='382416733822.dkr.ecr.us-east-1.amazonaws.com/xgboost:1.2-1',
    role=role,
    instance_count=1,
    instance_type=instance_type,
    output_path=model_output_path
)

# 定義訓練步驟
train_step = TrainingStep(
    name="TrainModel",
    estimator=estimator,
    inputs={
        'train': TrainingInput(s3_input_data, content_type="csv")
    }
)


In [26]:
from sagemaker.model import Model
from sagemaker.workflow.pipeline_context import PipelineSession

# 創建模型
model = Model(
    image_uri=estimator.image_uri,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    sagemaker_session=PipelineSession()
)

model

In [27]:
from sagemaker.workflow.model_step import ModelStep

# 定義模型部署步驟
deploy_step = ModelStep(
    name='ModelDeployment',
    step_args=model.create(instance_type="ml.m5.large"),
)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [29]:
# 定義 Pipeline
pipeline = Pipeline(
    name="TrainingAndDeploymentPipeline",
    parameters=[s3_input_data, instance_type, model_output_path],
    steps=[train_step, deploy_step]
)

# 開始執行 Pipeline
pipeline.upsert(role_arn=role)
# execution = pipeline.start()


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-1:070576557102:pipeline/TrainingAndDeploymentPipeline',
 'ResponseMetadata': {'RequestId': '516e9c96-7cc7-4fdd-a51c-2edd86858855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '516e9c96-7cc7-4fdd-a51c-2edd86858855',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Mon, 09 Sep 2024 23:26:39 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# 取得執行狀態
execution.describe()

# 等待 Pipeline 執行完成
execution.wait()
